<a href="https://colab.research.google.com/github/muhammadanas25/Stock_trend_prediction/blob/main/svm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Initial imports
import pandas as pd
!pip install path
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
# Initial imports
import pandas as pd
import numpy as np
from path import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

from sklearn.compose import make_column_selector as selector
 

In [3]:
# Loading data
#file_path = Path("HPQ.csv")
df = pd.read_csv("ORCL.csv")

df.dropna(inplace=True)

df.tail()

,Date,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend
749,23/06/2021,78.760002,78.870003,78.059998,78.099998,7295900,0.049384,-0.837993,Negative,Positive,Negative,0,Positive
750,24/06/2021,78.650002,78.959999,77.610001,77.739998,9624000,0.096294,-1.157029,Negative,Negative,Negative,0,Negative
751,25/06/2021,78.029999,78.610001,77.559998,78.459999,12845000,0.857142,0.551071,Positive,Negative,Positive,0,Negative
752,28/06/2021,78.489998,78.970001,78.330002,78.500000,5605400,0.265622,0.012743,Negative,Positive,Positive,0,Positive
753,29/06/2021,78.540001,78.589996,77.820000,78.099998,5392900,0.363636,-0.560227,Negative,Negative,Negative,0,Positive


In [4]:
appl_df = df.set_index("Date")
appl_df.tail()

,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend
Date,,,,,,,,,,,,
23/06/2021,78.760002,78.870003,78.059998,78.099998,7295900,0.049384,-0.837993,Negative,Positive,Negative,0,Positive
24/06/2021,78.650002,78.959999,77.610001,77.739998,9624000,0.096294,-1.157029,Negative,Negative,Negative,0,Negative
25/06/2021,78.029999,78.610001,77.559998,78.459999,12845000,0.857142,0.551071,Positive,Negative,Positive,0,Negative
28/06/2021,78.489998,78.970001,78.330002,78.500000,5605400,0.265622,0.012743,Negative,Positive,Positive,0,Positive
29/06/2021,78.540001,78.589996,77.820000,78.099998,5392900,0.363636,-0.560227,Negative,Negative,Negative,0,Positive


In [9]:
from sklearn.svm import SVC
new_df=appl_df
new_df
#new_df.drop(columns=['Twitter Sentiments'],inplace=True)

target=new_df['FutureTrend']
new_df=new_df.drop("FutureTrend",axis =1)
target= target.replace(['Negative','Positive',  'Neutral'],[0,1,2])
# new_df['diff']=new_df['Open']-new_df['Close']
# new_df.drop(columns=['Open', 'Close'],inplace=True)
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(new_df)
categorical_columns = categorical_columns_selector(new_df)

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard-scaler', numerical_preprocessor, numerical_columns)])
X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.2, random_state=42)
new_df=preprocessor.fit_transform(new_df)

model_svc = make_pipeline(preprocessor, SVC())
model_svc.fit(X_train,y_train)

  # Scoring the model

  # Scoring the model
print("Accuracy score (training): {0:.3f}".format(
        model_svc.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        model_svc.score(
            X_test, 
            y_test)))

Accuracy score (training): 0.625
Accuracy score (validation): 0.490


In [8]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']}
#new_df= new_df.replace(['Negative','Positive',  'Neutral'],[0,1,2])

In [10]:
 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.2, random_state=42)
# fitting the model for grid search
grid.fit(X_train, y_train)
print("Accuracy score (training): {0:.3f}".format(
       grid.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        grid.score(
            X_test, 
            y_test)))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.545 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.537 total time=   0.1s


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  UserWarning,


[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.537 total time=   0.1s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.542 total time=   0.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.542 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.562 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.562 total time=   0.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.612 total time=   0.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.608 total time=   0.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.533 total time=   0.1s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.545 total time=   0.1s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.537 total time=   0.1s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.537 total time=   0.1s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.542 total time=   0.1s
[CV 5/5] END ......C=0.1, ga

In [13]:
print (grid.best_params_)


{'C': 100, 'gamma': 1, 'kernel': 'linear'}
